Universidad Central de Venezuela  
Facultad de Ciencias  
Escuela de Computación  
Minería de Datos  

Nombre: Oscary Arocha  
C.I: 30.697.617 

<center><h1>Asignación #1: Pre-procesamiento</h1></center> 

Para esta asignación se usará la librería pandas para preprocesar el dataset World University Rankings 2023.

Lo primero es cargar el dataset y estandarizar los valores nulos que se encontraron.

In [169]:
import pandas as pd
import numpy as np

# Definir valores que deben ser tratados como nulos
valores_nulos = ["nan", "NaN", "null", "n/a", "N/A",'—', '-']

df = pd.read_csv('./WorldUniversityRankings2023.csv', na_values=valores_nulos)
df.head()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3


Como primer problema funcional, hay universidades que no tienen información en campos importantes como el ranking, el número de estudiantes y los ratios, incluso hay algunos registros sin nombre de la universidad. Esto nos deja sin información útil para el análisis por lo que se procede a eliminarlas para ahorrar recursos.

In [170]:
# Cantidad de registros antes de limpiar
print(f"Total de registros antes de limpiar: {len(df)} \n")

# Mostrar filas con NaN en 'University Rank'
nulos = df[df['University Rank'].isna()]
print(nulos[['University Rank', 'Name of University']])

# Calcular cuántas filas tienen NaN en 'University Rank'
total_nulos = df['University Rank'].isna().sum()
print(f"\nHay {total_nulos} filas con NaN en la columna 'University Rank'")

# Eliminar filas nulas de 'University Rank' 
df = df.dropna(subset=['University Rank']) 

# Verificar cuántos registros quedaron
print(f"Total de registros después de limpiar: {len(df)}")


Total de registros antes de limpiar: 2341 

     University Rank                   Name of University
2209             NaN                                  NaN
2210             NaN                                  NaN
2211             NaN                                  NaN
2212             NaN                                  NaN
2213             NaN                                  NaN
...              ...                                  ...
2336             NaN   University of the West of Scotland
2337             NaN                University of Windsor
2338             NaN          University of Wolverhampton
2339             NaN              University of Wuppertal
2340             NaN  Xi’an Jiaotong-Liverpool University

[132 rows x 2 columns]

Hay 132 filas con NaN en la columna 'University Rank'
Total de registros después de limpiar: 2209


## Problemas Estructurales:

1. Lo primero es que la columna "University Rank" contiene multiples valores ya que hay universidades cuyo ranking está entre dos valores, para solucionarlo se descompone en dos columnas: rank_min y rank_max. 

    Además, existen valores como "1501+" que indican que el ranking es igual o mayor a 1501, en estos casos, el valor asignado a rank_max será el número máximo de universidades presentes en el ranking. Aunque se busca que los datos sean usables, es importante reconocer que esta decisión introduce cierto sesgo, ya que no se cuenta con el valor máximo real del rango para estas universidades.

    1.1. Se decidió crear una variable 'Is_reporter' para preservar la información original, ya que para normalizar a las observaciones que tienen el valor "Reporter" se les debe asignar NaN, sin embargo, se debe indicar si dicha universidades participaron en el ranking y no se les dió una calificación, o si simplemente es un dato perdido.

In [171]:
# Crear la columna indicadora antes de convertir 'Reporter' a NaN
df['Is_reporter'] = df['University Rank'] == 'Reporter'

# Reemplazar 'Reporter' con NaN y normalizar guiones
df['University Rank'] = df['University Rank'].replace('Reporter', np.nan)
df['University Rank'] = df['University Rank'].str.replace('–', '-').str.replace('—', '-')

# Reemplazar "+" por un rango con el máximo
max_rank = len(df)
df['University Rank'] = df['University Rank'].str.replace('+', f'-{max_rank}')

# Dividir por el guión
df[['Rank_min', 'Rank_max']] = df['University Rank'].str.split('-', expand=True)

# Si rank_max es nulo, usar rank_min
df['Rank_max'] = df['Rank_max'].fillna(df['Rank_min'])

# Convertir a numérico
df['Rank_min'] = pd.to_numeric(df['Rank_min'], errors='coerce').astype('Int64')
df['Rank_max'] = pd.to_numeric(df['Rank_max'], errors='coerce').astype('Int64')


# Reordenar columnas
cols = list(df.columns)
cols.insert(1, cols.pop(cols.index('Rank_min')))
cols.insert(2, cols.pop(cols.index('Rank_max')))
cols.insert(3, cols.pop(cols.index('Is_reporter')))
df = df[cols]

print(df[['University Rank', 'Rank_min', 'Rank_max', 'Is_reporter']].sample(10))

df.sample(5)


     University Rank  Rank_min  Rank_max  Is_reporter
751          601-800       601       800        False
385          351-400       351       400        False
1266       1201-1500      1201      1500        False
748          601-800       601       800        False
1277       1201-1500      1201      1500        False
1479       1501-2209      1501      2209        False
13                14        14        14        False
693          601-800       601       800        False
610          601-800       601       800        False
661          601-800       601       800        False


,University Rank,Rank_min,Rank_max,Is_reporter,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
442,401-500,401,500,False,Koç University,Turkey,"5,882",19.7,8%,48 : 52,42.1–44.9,30.7,42.3,56.1,100.0,49.3
1860,NaN,<NA>,<NA>,True,"Federal University of Technology, Owerri",Nigeria,"22,610",18.7,0%,31 : 69,NaN,NaN,NaN,NaN,NaN,NaN
1712,NaN,<NA>,<NA>,True,Al-Ayen University,Iraq,"11,000",51.2,0%,66 : 34,NaN,NaN,NaN,NaN,NaN,NaN
954,801-1000,801,1000,False,Yangzhou University,China,"31,066",13.3,11%,58 : 42,29.8–33.9,23.9,18.4,51.6,71.9,38.6
697,601-800,601,800,False,University of Limerick,NaN,"14,733",25.1,20%,50 : 50,34.0–39.2,24.2,30.3,50.8,40.7,83.0



2. El mismo problema ocurre en la columna "OverAll Score" hay registros que representan un valor único y otros que representan un rango, entonces de igual manera se separa en dos columnas: Score_Min y Score_Max, para los nulos o n/a se transforman en NaN.


In [ ]:

# Normalizar diferentes tipos de guiones
df['OverAll Score'] = df['OverAll Score'].str.replace('–', '-').str.replace('—', '-')

# Dividir columna por el guión
df[['Score_Min', 'Score_Max']] = df['OverAll Score'].str.split('-', expand=True)

# Si Score_Max es nulo, usar Score_Min
df['Score_Max'] = df['Score_Max'].fillna(df['Score_Min'])

# Convertir a numérico
df['Score_Min'] = pd.to_numeric(df['Score_Min'], errors='coerce')
df['Score_Max'] = pd.to_numeric(df['Score_Max'], errors='coerce')


# Verificar columnas
df[['University Rank', 'Name of University', 'OverAll Score', 'Score_Min', 'Score_Max']].sample(10)


,University Rank,Name of University,OverAll Score,Score_Min,Score_Max
1317,1201-1500,Nicolaus Copernicus University in Toruń,18.4-24.3,18.4,24.3
380,351-400,University of New Mexico (Main campus),45.0-46.9,45.0,46.9
2175,NaN,Üsküdar University,NaN,NaN,NaN
1284,1201-1500,Kyushu Institute of Technology (Kyutech),18.4-24.3,18.4,24.3
228,201-250,"St George’s, University of London",51.2-54.3,51.2,54.3
1059,1001-1200,University of León,24.4-29.7,24.4,29.7
1623,1501-2209,Universidad Panamericana (UP),10.4-18.3,10.4,18.3
84,85,McMaster University,65.1,65.1,65.1
1390,1201-1500,Institut Teknologi Sepuluh Nopember,18.4-24.3,18.4,24.3
2137,NaN,Tashkent Institute of Irrigation and Agricultu...,NaN,NaN,NaN



3. En la columna Female:Male Ratio también se tienen multiples variables por registro, se debe hacer una limpieza de los espacios y luego descomponer en female_ratio y male_ratio, de esta forma se mejora la usabilidad de la columna y es más facil aplicar técnicas de DM.




In [ ]:
# Dividr la columna usando el separador ':'
df[['Female_ratio', 'Male_ratio']] = df['Female:Male Ratio'].str.split(':', expand=True)

# Usamos .str.strip() para eliminar los espacios en blanco que sobran
df['Female_ratio'] = df['Female_ratio'].str.strip()
df['Male_ratio'] = df['Male_ratio'].str.strip()

# Convertirlos a números 
df['Female_ratio'] = pd.to_numeric(df['Female_ratio'], errors='coerce')
df['Male_ratio'] = pd.to_numeric(df['Male_ratio'], errors='coerce')

# Reordenar columnas
cols = list(df.columns)
cols.insert(9, cols.pop(cols.index('Female_ratio')))
cols.insert(10, cols.pop(cols.index('Male_ratio')))
df = df[cols]

df[['University Rank', 'Name of University', 'Female:Male Ratio','Female_ratio', 'Male_ratio']].sample(10)
#df.sample(5)

,University Rank,Name of University,Female:Male Ratio,Female_ratio,Male_ratio
1008,1001-1200,University of Extremadura,56 : 44,56.0,44.0
0,1,University of Oxford,48 : 52,48.0,52.0
361,351-400,Golestan University of Medical Sciences,60 : 40,60.0,40.0
836,801-1000,Karlstad University,63 : 37,63.0,37.0
1670,1501-2209,Universiti Teknologi MARA,67 : 33,67.0,33.0
1947,NaN,Kyiv National Economic University,61 : 39,61.0,39.0
1546,1501-2209,Universidad Industrial de Santander (UIS),45 : 55,45.0,55.0
1792,NaN,University of Celaya,50 : 50,50.0,50.0
802,801-1000,University of Clermont Auvergne,57 : 43,57.0,43.0
1417,1201-1500,University of Valladolid,57 : 43,57.0,43.0


## Problemas Funcionales:

Además de los problemas detectados al inicio de los datos faltantes en el nombre, el ranking, el número de estudiantes y los ratios, se tiene:



1. La columna "International Student" se entiende como una variable de proporción y para evitar mezclar números con el caracter %, se procede a limpiar y normalizar, eliminando el caracter y transformando el campo a tipo númerico.

In [174]:
# Renombrar la columna para mayor claridad
df = df.rename(columns={'International Student': 'International Student (%)'})

# Eliminar el símbolo de '%' y cualquier espacio en blanco
df['International Student (%)'] = df['International Student (%)'].str.replace('%', '').str.strip()

# Convertimos a numérico 
df['International Student (%)'] = pd.to_numeric(df['International Student (%)'], errors='coerce')

df.sample(5)


,University Rank,Rank_min,Rank_max,Is_reporter,Name of University,Location,No of student,No of student per staff,International Student (%),Female_ratio,Male_ratio,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Score_Min,Score_Max
1402,1201-1500,1201,1500,False,Tokyo University of Science,Japan,"18,091",24.3,3.0,25.0,75.0,25 : 75,18.4-24.3,22.3,24.4,11.3,47.2,24.5,18.4,24.3
6,7,7,7,False,Princeton University,United States,"8,279",8.0,23.0,46.0,54.0,46 : 54,92.4,87.6,95.9,99.1,66.0,80.3,92.4,92.4
1500,1501-2209,1501,2209,False,Erzincan Binali Yıldırım University,Turkey,"13,773",18.7,3.0,54.0,46.0,54 : 46,10.4-18.3,13.6,8.2,31.5,36.9,18.3,10.4,18.3
2026,NaN,<NA>,<NA>,True,Navoi State Pedagogical Institute,Uzbekistan,"12,791",26.4,0.0,72.0,28.0,72 : 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1706,NaN,<NA>,<NA>,True,Air University,Pakistan,"6,404",15.2,0.0,32.0,68.0,32 : 68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Después de hacer esto, si filtramos por las filas que quedaron en NaN, vemos 
que hay dos registros que tienen varias variables vacías. Considero más 
conveniente eliminarlas.

In [175]:
df[df['International Student (%)'].isna()]

,University Rank,Rank_min,Rank_max,Is_reporter,Name of University,Location,No of student,No of student per staff,International Student (%),Female_ratio,Male_ratio,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Score_Min,Score_Max
836,801-1000,801,1000,False,Karlstad University,Sweden,"9,698",14.0,NaN,63.0,37.0,63 : 37,29.8-33.9,17.3,14.4,54.6,37.5,41.2,29.8,33.9
1758,NaN,<NA>,<NA>,True,Balochistan University of Engineering and Tech...,Pakistan,"2,990",15.0,NaN,7.0,93.0,7 : 93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1865,NaN,<NA>,<NA>,True,Fundação Oswaldo Cruz,Brazil,"3,699",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
# Eliminar filas donde todas las columnas especificadas son NaN
df.dropna(subset=['International Student (%)', 'OverAll Score', 'Teaching Score', 'Research Score'], how='all' ,inplace=True)

# Verificar que se eliminaron las filas correctas
df[df['International Student (%)'].isna()]


,University Rank,Rank_min,Rank_max,Is_reporter,Name of University,Location,No of student,No of student per staff,International Student (%),Female_ratio,Male_ratio,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Score_Min,Score_Max
836,801-1000,801,1000,False,Karlstad University,Sweden,"9,698",14.0,NaN,63.0,37.0,63 : 37,29.8-33.9,17.3,14.4,54.6,37.5,41.2,29.8,33.9


**Entonces el dataset resultante sería:**

In [177]:
df.drop(columns=['University Rank', 'Female:Male Ratio', 'OverAll Score'], inplace=True)

df.sample(10)

,Rank_min,Rank_max,Is_reporter,Name of University,Location,No of student,No of student per staff,International Student (%),Female_ratio,Male_ratio,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Score_Min,Score_Max
536,501,600,False,Johannes Kepler University of Linz,Austria,"7,133",22.5,13.0,50.0,50.0,32.5,32.9,42.3,73.1,70.4,39.3,42.0
1164,1201,1500,False,AGH University of Krakow,Poland,"20,240",12.7,3.0,35.0,65.0,20.6,18.6,18.7,56.8,25.5,18.4,24.3
1580,1501,2209,False,Mapúa University,NaN,"6,576",23.9,1.0,36.0,64.0,13.8,13.5,8.5,36.9,16.9,10.4,18.3
447,401,500,False,University of Lincoln,United Kingdom,"14,480",15.7,13.0,55.0,45.0,17.8,17.4,85.8,37.5,75.4,42.1,44.9
101,101,101,False,King Abdulaziz University,NaN,"31,545",7.8,22.0,54.0,46.0,51.7,36.7,91.9,74.2,93.6,63.0,63.0
993,1001,1200,False,China Pharmaceutical University,China,"24,270",22.4,2.0,69.0,31.0,18.8,15.8,41.8,57.8,18.7,24.4,29.7
1329,1201,1500,False,University of Ostrava,Czech Republic,"8,492",16.4,11.0,69.0,31.0,18.5,15.1,16.2,37.1,46.0,18.4,24.3
823,801,1000,False,Guangdong University of Technology,China,"46,040",18.1,0.0,29.0,71.0,16.2,14.8,69.9,61.1,25.1,29.8,33.9
1307,1201,1500,False,Nagoya City University,Japan,"4,514",7.5,4.0,48.0,52.0,24.5,15.8,24.1,51.2,21.7,18.4,24.3
857,801,1000,False,Lovely Professional University,NaN,"33,469",14.8,6.0,24.0,76.0,18.2,10.6,66.8,38.1,28.9,29.8,33.9
